In [ ]:
dataset_size = len(filenames)
dataset = torch.zeros(dataset_size - 4, 15, 128, 128)
for i in range(dataset_size-4):
 t_minus_2 = torchvision.io.read_image(os.path.join(data_dir, filenames[i]))
 t_minus_1 = torchvision.io.read_image(os.path.join(data_dir, filenames[i+1]))
 t = torchvision.io.read_image(os.path.join(data_dir, filenames[i+2]))
 t_plus_1 = torchvision.io.read_image(os.path.join(data_dir, filenames[i+3]))
 t_plus_2 = torchvision.io.read_image(os.path.join(data_dir, filenames[i+4]))

 dataset[i] = torch.cat((t_minus_2, t_minus_1, t ,t_plus_1, t_plus_2))

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        # 3 is input channel because of RGB * 5 images. 
        # 128 is the output channel or learnable filters
        # Kernel size 15 is temporal kernel width (3 RGB channel * 5 images)
        # (1*1) is spatial kernel width
        self.conv3d = nn.Conv3d(15, 128, kernel_size=(1, 1, 1))
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv3d(x)
        x = self.relu(x)
        return x

In [ ]:
model = MyModel()